#### This notebook is an analyzed version of https://github.com/hardikbansal/CycleGAN

## Imort module

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import imageio 
import os
import shutil
from PIL import Image
import time
import random
import sys

## Import .layers

### Leaky Relu

In [ ]:
def lrelu(x, leak=0.2, name="lrelu", alt_relu_impl=False):

    with tf.variable_scope(name):
        if alt_relu_impl:
            f1 = 0.5 * (1 + leak)
            f2 = 0.5 * (1 - leak)
            # lrelu = 1/2 * (1 + leak) * x + 1/2 * (1 - leak) * |x|
            return f1 * x + f2 * abs(x)
        else:
            return tf.maximum(x, leak*x)

- `tf.variable_scope()` : tf.get_variable() 을 이용하여 기존에 선언된 global variable 들을 재사용하기 위한 용도 (name 으로 group 지음)

    - `tf.Variable()` : 언제나 새로운 객체 (변수) 를 생성
    
    - `tf.get_variable()` : 이미 존재하는 객체를 매개변수로 받을 수도 있음 $\rightarrow$ 변수 재사용 가능
    
    (ref: [Tensorflow 기초](https://datascienceschool.net/view-notebook/5cbab09d777841f591a67928d7043f51/))
    

- `alternative_relu_impl` (alter_relu_implemetation) : custom version of Leaky ReLU

    - leak = 0.2 $\rightarrow$ f1 = 0.6, f2 =0.4
    
    - `f1 * x + f2 * abs(x)`
        
        - $x>0$ : x
        
        - $x\le0$ : 0.2 * x
        
        
- `tf.maximum(x, leak*x)` : basic Leaky ReLU
    
    - $x>0$ : x
        
    - $x\le0$ : 0.2 * x




### Instance Norm (Batch Normalization)

In [ ]:
def instance_norm(x):

    with tf.variable_scope("instance_norm"):
        epsilon = 1e-5
        mean, var = tf.nn.moments(x, [1, 2], keep_dims=True)
        
        scale = tf.get_variable('scale',[x.get_shape()[-1]], 
            initializer=tf.truncated_normal_initializer(mean=1.0, stddev=0.02))
        
        offset = tf.get_variable('offset',[x.get_shape()[-1]],initializer=tf.constant_initializer(0.0))
        
        out = scale*tf.div(x-mean, tf.sqrt(var+epsilon)) + offset

        return out

- `tf.nn.moments(array, shape)` : 정해진 axis 에 맞춰서 평균 (mean), 분산 (variance) 를 계산

```python
import tensorflow as tf

x = tf.constant([[1,2],[3,4],[5,6]], dtype=tf.float32)
mean, variance = tf.nn.moments(x, [0])
with tf.Session() as sess:
    m, v = sess.run([mean, variance])
    print(m, v)
>>> [ 3.  4.] [ 2.66666675  2.66666675]
```

>- [ 3.  4.] : means of axis 0 $\rightarrow$ row
>
>    - 3. : mean of [1, 3, 5]
>
>    - 4. : mean of [2, 4, 6]
>
>
>- [ 2.66666675  2.66666675] : variances of axis 0 $\rightarrow$ row
>
>    - 2.66666675 : variance of [1, 3, 5]
>
>    - 2.66666675 : variance of [2, 4, 6]


- `scale` : multiply scaler for distribution

    - change **shape** of distribution
    
    - $scale \times N(\mu, \sigma^2)$
    

- `offset` : add intercept for distribution 

    - change **location** of distribution
    
    - $N(\mu, \sigma^2) + offset$
    

- `out` : change both location and shape of distribution

    - $scale \times N(\mu, \sigma^2) + offset$
    
        - **[location]** $\mu \rightarrow scale \times \mu + offset$
        
        - **[shape]** $\sigma \rightarrow scale \times \sigma$

### Convolution layer

In [ ]:
def general_conv2d(inputconv, o_d=64, f_h=7, f_w=7, s_h=1, s_w=1, stddev=0.02, 
                   padding="VALID", name="conv2d", do_norm=True, do_relu=True, relufactor=0):
    with tf.variable_scope(name):
        
        conv = tf.contrib.layers.conv2d(inputconv, o_d, f_w, s_w, padding, activation_fn=None,
                                        weights_initializer=tf.truncated_normal_initializer(stddev=stddev),
                                        biases_initializer=tf.constant_initializer(0.0))
        if do_norm:
            conv = instance_norm(conv)
#             conv = tf.contrib.layers.batch_norm(conv, decay=0.9, updates_collections=None, 
#                                                 epsilon=1e-5, scale=True, scope="batch_norm")
            
        if do_relu:
            if(relufactor == 0):
                conv = tf.nn.relu(conv,"relu")
            else:
                conv = lrelu(conv, relufactor, "lrelu")

        return conv

- **arguments**

>`inputconv` : previous layer's output
>
>`o_d` : number of filters
>   
>`f_h` , `f_w` : fiter height & width
>   
>`s_h` , `s_w` : stride height & width
>    
>`stddev` : standard deviation for weight initializer
>    
>`padding` : 'VALID' (0 padding) or 'SAME'
>    
>`do_norm` : check if batch normalization will be conducted
>    
>`do_relu` : check if ReLU activation function will be used
>    
>`relufactor` : factor for Leaky ReLU (basic ReLU if 0)



- `tf.contrib.layers.conv2d()`

    - `inputconv` : input vector
    
    - `o_d` : number of filters
    
    - `f_w` : filter size (width == height)
    
    - `s_h` : stride size (width == height)
    
    - `activation_fn` : define activation function (defined as `None` due to batch normalization)
    
    - `weights_initializer`
    
        - `tf.truncated_normal_initializer(stddev)` : use truncated normal distribution for initialization  
       
    - `biases_initializer`
    
        - `tf.constant_initializer(0.0)` : initialize bias as constant (better be initializing as 0)
    
    

### De-Convolutional layer

In [ ]:
def general_deconv2d(inputconv, outshape, o_d=64, f_h=7, f_w=7, s_h=1, s_w=1, stddev=0.02, 
                     padding="VALID", name="deconv2d", do_norm=True, do_relu=True, relufactor=0):
    with tf.variable_scope(name):

        conv = tf.contrib.layers.conv2d_transpose(inputconv, o_d, [f_h, f_w], [s_h, s_w], padding, 
                                                  activation_fn=None,
                                                  weights_initializer=
                                                  tf.truncated_normal_initializer(stddev=stddev),
                                                  biases_initializer=tf.constant_initializer(0.0))
        
        if do_norm:
            conv = instance_norm(conv)
#             conv = tf.contrib.layers.batch_norm(conv, decay=0.9, updates_collections=None, 
#                                                 epsilon=1e-5, scale=True, scope="batch_norm")
            
        if do_relu:
            if(relufactor == 0):
                conv = tf.nn.relu(conv,"relu")
            else:
                conv = lrelu(conv, relufactor, "lrelu")

        return conv

- **arguments**

> `outshape` : define output shape (NOT USED)
    


- `tf.contrib.layers.conv2d_transpose()`

    - **<span style="color: blue">transpose</span>**

    - `o_d` : output dimension (number of filters)

    - `[f_h, f_w]` : shape of filter
    
    - `[s_h, s_w]` : shape for stride



## Import .model

### constants

- `img_height = 256` : height of input image

- `img_width = 256` : width of input image
    
- `img_layer = 3` : channels (RGB) of input image

- `img_size = img_height * img_width`

- `batch_size = 1` : mini batch size??

- `pool_size = 50` : number of pooling filters

- `ngf = 32` : number of filters for generator $G$

- `ndf = 64` : numbe of filters for discriminator $D$


### ResNet block

In [ ]:
def build_resnet_block(inputres, dim, name="resnet"):
    
    with tf.variable_scope(name):

        out_res = tf.pad(inputres, [[0, 0], [1, 1], [1, 1], [0, 0]], "REFLECT")
        out_res = general_conv2d(out_res, dim, 3, 3, 1, 1, 0.02, "VALID","c1")
        out_res = tf.pad(out_res, [[0, 0], [1, 1], [1, 1], [0, 0]], "REFLECT")
        out_res = general_conv2d(out_res, dim, 3, 3, 1, 1, 0.02, "VALID","c2",do_relu=False)
        
        return tf.nn.relu(out_res + inputres)

- **arguments**

> `inputres` : output of previous resnet block
>
> `dim` : filter size for convolution layer



- `tf.pad(tensor, paddings, mode='CONSTANT')`

```python
t = tf.constant([[1, 2, 3], [4, 5, 6]])
paddings = tf.constant([[1, 1,], [2, 2]])

tf.pad(t, paddings, "CONSTANT")  # [[0, 0, 0, 0, 0, 0, 0],
                                 #  [0, 0, 1, 2, 3, 0, 0],
                                 #  [0, 0, 4, 5, 6, 0, 0],
                                 #  [0, 0, 0, 0, 0, 0, 0]]

tf.pad(t, paddings, "REFLECT")  # [[6, 5, 4, 5, 6, 5, 4],
                                #  [3, 2, 1, 2, 3, 2, 1],
                                #  [6, 5, 4, 5, 6, 5, 4],
                                #  [3, 2, 1, 2, 3, 2, 1]]
```

> - padding will be added to this two dimensional array
>
> ```python
> t = tf.constant([[1, 2, 3], [4, 5, 6]])
>
> [[1, 2, 3] 
>  [4, 5, 6]]
> ```
> 
> - `paddings = tf.constant([[1, 1,], [2, 2]])`
>
>     - [padding_**top**, padding_**bottom**], [padding_**left**, padding_**right**] : 위, 아래, 양옆 으로 얼마만큼 붙일지 설정
>
>
> - `mode`
>
>     - 어떤 숫자를 padding 할지 방법 설정
>
>     - 'CONSTANT' : 정해진 숫자 하나로 채움 (default = 0) ([0, 0, <span style="color: blue">1, 2, 3</span>, 0, 0])
>    
>     - 'REFLECT' : 기존 데이터의 패턴을 반복하는 방식 ([3, 2, <span style="color: blue">1, 2, 3</span>, 2, 1])



- `general_conv2d()` : user defined

### Generator $G$ : ResNet block $\times$ 6

In [ ]:
def build_generator_resnet_6blocks(inputgen, name="generator"):
    with tf.variable_scope(name):
        f = 7
        ks = 3
        
        pad_input = tf.pad(inputgen,[[0, 0], [ks, ks], [ks, ks], [0, 0]], "REFLECT")
        o_c1 = general_conv2d(pad_input, ngf, f, f, 1, 1, 0.02,name="c1")
        o_c2 = general_conv2d(o_c1, ngf*2, ks, ks, 2, 2, 0.02,"SAME","c2")
        o_c3 = general_conv2d(o_c2, ngf*4, ks, ks, 2, 2, 0.02,"SAME","c3")

        o_r1 = build_resnet_block(o_c3, ngf*4, "r1")
        o_r2 = build_resnet_block(o_r1, ngf*4, "r2")
        o_r3 = build_resnet_block(o_r2, ngf*4, "r3")
        o_r4 = build_resnet_block(o_r3, ngf*4, "r4")
        o_r5 = build_resnet_block(o_r4, ngf*4, "r5")
        o_r6 = build_resnet_block(o_r5, ngf*4, "r6")

        o_c4 = general_deconv2d(o_r6, [batch_size,64,64,ngf*2], ngf*2, ks, ks, 2, 2, 0.02,"SAME","c4")
        o_c5 = general_deconv2d(o_c4, [batch_size,128,128,ngf], ngf, ks, ks, 2, 2, 0.02,"SAME","c5")
        o_c5_pad = tf.pad(o_c5,[[0, 0], [ks, ks], [ks, ks], [0, 0]], "REFLECT")
        o_c6 = general_conv2d(o_c5_pad, img_layer, f, f, 1, 1, 0.02,"VALID","c6",do_relu=False)

        # Adding the tanh layer

        out_gen = tf.nn.tanh(o_c6,"t1")


        return out_gen

- **constants**

> `f` : input layer filter size (width == height)
>
> `ks` : padding size or filter size (width == height)



- `tf.pad(inputgen,[[0, 0], [ks, ks], [ks, ks], [0, 0]]`

    - [number of data, image **height**, image **width**, channels]
    
        - do not append padding on 'number of data' (dim = 0) and 'channels' (dim = 3) $\rightarrow$ [0, 0]
        
        - only append padding for image it self (<span style="color: blue">height (dim = 1)</span>, <span style="color: red">width (dim = 2)</span>) $\rightarrow$ <span style="color: blue">[top, bottom]</span>, <span style="color: red">[left, right]</span>



- `general_deconv2d(input vector, output_shape, num_filters, ...)`

    - `output_shape = [batch_size,64,64,ngf*2]` : [channel, width, height, num_filters]
    
        - the order of shape is **reveresed** because of <span style="color: blue">**conv2d_transpose**</span> (De-Conv layer)



- all hidden layers use **Leaky ReLU** with factor = 0.02



- **architecture**

> - Padding
>
> - input Conv layer
>
> - Conv layer $\times$ 2
>
> - **ResNet block $\times$ 6**
>
> - De-Conv layer $\times$ 2
>
> - Padding
>
> - Conv layer
>
> - Tanh Activation



### Generator $G$ : ResNet block $\times$ 9

In [ ]:
def build_generator_resnet_9blocks(inputgen, name="generator"):
    with tf.variable_scope(name):
        f = 7
        ks = 3
        
        pad_input = tf.pad(inputgen,[[0, 0], [ks, ks], [ks, ks], [0, 0]], "REFLECT")
        o_c1 = general_conv2d(pad_input, ngf, f, f, 1, 1, 0.02,name="c1")
        o_c2 = general_conv2d(o_c1, ngf*2, ks, ks, 2, 2, 0.02,"SAME","c2")
        o_c3 = general_conv2d(o_c2, ngf*4, ks, ks, 2, 2, 0.02,"SAME","c3")

        o_r1 = build_resnet_block(o_c3, ngf*4, "r1")
        o_r2 = build_resnet_block(o_r1, ngf*4, "r2")
        o_r3 = build_resnet_block(o_r2, ngf*4, "r3")
        o_r4 = build_resnet_block(o_r3, ngf*4, "r4")
        o_r5 = build_resnet_block(o_r4, ngf*4, "r5")
        o_r6 = build_resnet_block(o_r5, ngf*4, "r6")
        o_r7 = build_resnet_block(o_r6, ngf*4, "r7")
        o_r8 = build_resnet_block(o_r7, ngf*4, "r8")
        o_r9 = build_resnet_block(o_r8, ngf*4, "r9")

        o_c4 = general_deconv2d(o_r9, [batch_size,128,128,ngf*2], ngf*2, ks, ks, 2, 2, 0.02,"SAME","c4")
        o_c5 = general_deconv2d(o_c4, [batch_size,256,256,ngf], ngf, ks, ks, 2, 2, 0.02,"SAME","c5")
        o_c6 = general_conv2d(o_c5, img_layer, f, f, 1, 1, 0.02,"SAME","c6",do_relu=False)

        # Adding the tanh layer

        out_gen = tf.nn.tanh(o_c6,"t1")


        return out_gen

- **constants**

> `f` : input layer filter size (width == height)
>
> `ks` : padding size or filter size (width == height)


        
- **architecture**

> - Padding
>
> - input Conv layer
>
> - Conv layer $\times$ 2
>
> - **ResNet block $\times$ 9**
>
> - De-Conv layer $\times$ 2
>
> - Padding
>
> - Conv layer
>
> - Tanh Activation



### Discriminator $D$

In [ ]:
def build_gen_discriminator(inputdisc, name="discriminator"):
    
    with tf.variable_scope(name):
        f = 4

        o_c1 = general_conv2d(inputdisc, ndf, f, f, 2, 2, 0.02, "SAME", "c1", do_norm=False, relufactor=0.2)
        o_c2 = general_conv2d(o_c1, ndf*2, f, f, 2, 2, 0.02, "SAME", "c2", relufactor=0.2)
        o_c3 = general_conv2d(o_c2, ndf*4, f, f, 2, 2, 0.02, "SAME", "c3", relufactor=0.2)
        o_c4 = general_conv2d(o_c3, ndf*8, f, f, 1, 1, 0.02, "SAME", "c4",relufactor=0.2)
        o_c5 = general_conv2d(o_c4, 1, f, f, 1, 1, 0.02, "SAME", "c5",do_norm=False,do_relu=False)

        return o_c5

- **constants**

> `f` : filter size (width == height)

        
- **architecture**

> - input Conv layer
>
> - Conv layer $\times$ 4


### Discriminator $D$ with patched input

In [ ]:
def patch_discriminator(inputdisc, name="discriminator"):

    with tf.variable_scope(name):
        f= 4

        patch_input = tf.random_crop(inputdisc,[1,70,70,3])
        o_c1 = general_conv2d(patch_input, ndf, f, f, 2, 2, 0.02, "SAME", "c1", do_norm="False", relufactor=0.2)
        o_c2 = general_conv2d(o_c1, ndf*2, f, f, 2, 2, 0.02, "SAME", "c2", relufactor=0.2)
        o_c3 = general_conv2d(o_c2, ndf*4, f, f, 2, 2, 0.02, "SAME", "c3", relufactor=0.2)
        o_c4 = general_conv2d(o_c3, ndf*8, f, f, 2, 2, 0.02, "SAME", "c4", relufactor=0.2)
        o_c5 = general_conv2d(o_c4, 1, f, f, 1, 1, 0.02, "SAME", "c5",do_norm=False,do_relu=False)

        return o_c5

- **constants**

> `f` : filter size (width == height)

        
- **architecture**

> - input Conv layer
>
> - Conv layer $\times$ 4



- `tf.random_crop(value, size)`

    - value : image vector
    
    - size : wanted size after cropping
    
        - e.g. [32, 32, 3] 이미지에서 [24, 24, 3] 만큼을 자르고 싶을 때에는 size=[24, 24, 3]




## Import .main

### constants

- `img_height = 256` : height of image

- `img_width = 256` : width of image

- `img_layer = 3` : channels of image

- `img_size = img_height * img_width` : size of image

---

- `to_train = True` : check if training

- `to_test = False` : check if testing

- `to_restore = False` : check if training saved model continuously

- `output_path = "./output"`

- `check_dir = "./output/checkpoints/"`

- temp_check = 0

---

- `max_epoch = 1`

- `max_images = 100`

---

- `h1_size = 150` : (NOT USED)

- `h2_size = 300` : (NOT USED)

- `z_size = 100` : (NOT USED)

- `batch_size = 1` : mini-batch size

- `pool_size = 50` : number of pooling filters

- `sample_size = 10` : (NOT USED)

- `save_training_images = True` : check if save training images

- `ngf = 32` : number of filters for generator $G$

- `ndf = 64` : number of filters for discriminator $D$


## class CycleGAN():

### setup input images

In [ ]:
def input_setup(self):

        filenames_A = tf.train.match_filenames_once("./input/horse2zebra/trainA/*.jpg")    
        self.queue_length_A = tf.size(filenames_A)
        filenames_B = tf.train.match_filenames_once("./input/horse2zebra/trainB/*.jpg")    
        self.queue_length_B = tf.size(filenames_B)
        
        filename_queue_A = tf.train.string_input_producer(filenames_A)
        filename_queue_B = tf.train.string_input_producer(filenames_B)

        image_reader = tf.WholeFileReader()
        _, image_file_A = image_reader.read(filename_queue_A)
        _, image_file_B = image_reader.read(filename_queue_B)

        self.image_A = tf.subtract(tf.div(tf.image.resize_images(
            tf.image.decode_jpeg(image_file_A),[256,256]),127.5),1)
        self.image_B = tf.subtract(tf.div(tf.image.resize_images(
            tf.image.decode_jpeg(image_file_B),[256,256]),127.5),1)

> dataset ***A*** : horse image dataset
>
> dataset ***B*** : zebra image dataset



- `tf.train.match_filenames_once(*.*)` : read all file names in specific directory with specific extension

   - takes the list of all training images


    
- `tf.size()` : get length of the name list



- `tf.io.decode_jpeg(contents, channels=0, ratio=1)` : decode a JPEG-encoded image to a uint8 tensor

    - channels
    
        - 0 : use the number of channels in the JPEG-encoded image
        
        - 1 : output a grayscale image
        
        - 3 : output an RGB image
        
    - ratio : downscaling the image by an integer factor during decoding. Allowed values are: 1, 2, 4, and 8



- `self.image_A` / `self.image_B` : Normalize image vector

    - `tf.image.resize_images(decoded_image, [256,256])` : resize image to 256 $\times$ 256
    
    - `tf.div(resized_image, 127.5)` : divide each pixel value by a half of the width (256 / 127.5 = 2)
    
    - `tf.subtract(divided_image, 1)` : subtract each pixel value by 1
    
    $\Rightarrow$ now all pixels of the image have value **range near 1**


### 2 GANs

${L_{GAN}(G(x), y) + ||F(G(x)) - x||_{1}} + {L_{GAN}(F(y), x) + ||G(F(y)) - y||_{1}}$

>- Generator 1 $G$
>
>    - $x$: horse
>
>    - $G$: horse $\rightarrow$ zebra
>
>    - $G(x)$: **zebra** image generated by $G$
>
>
>- Generator 2 $F$
>
>    - $y$: zebra
>
>    - $F$: zebra $\rightarrow$ horse
>
>    - $F(y)$: **horse** image generated by $F$


**[Model *A*] horse $\rightarrow$ zebra 모델**

- $G$ 가 horse 를 만들어내는 역할 (== **A**)
    
- $F$ 가 zebra 로 되돌리는 역할 (== **B**)
    
    
**inversely...**


**[Model *B*] zebra $\rightarrow$ horse 모델**

- $F$ 가 zebra 를 만들어내는 역할 (== **B**)
    
- $G$ 가 horse 로 되돌리는 역할 (== **A**)

### read input images

In [ ]:
def input_read(self, sess):

        # Loading images into the tensors
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(coord=coord)

        num_files_A = sess.run(self.queue_length_A)
        num_files_B = sess.run(self.queue_length_B)

        self.fake_images_A = np.zeros((pool_size, 1, img_height, img_width, img_layer))
        self.fake_images_B = np.zeros((pool_size, 1, img_height, img_width, img_layer))


        self.A_input = np.zeros((max_images, batch_size, img_height, img_width, img_layer))
        self.B_input = np.zeros((max_images, batch_size, img_height, img_width, img_layer))

        for i in range(max_images): 
            image_tensor = sess.run(self.image_A)
            if(image_tensor.size == img_size*batch_size*img_layer):
                self.A_input[i] = image_tensor.reshape((batch_size, img_height, img_width, img_layer))

        for i in range(max_images):
            image_tensor = sess.run(self.image_B)
            if(image_tensor.size == img_size*batch_size*img_layer):
                self.B_input[i] = image_tensor.reshape((batch_size, img_height, img_width, img_layer))


        coord.request_stop()
        coord.join(threads)

- `self.fake_images_A` / `self.fake_images_B` : **random noise** input for generator A and B



- `self.A_input` / `self.B_input` : stores all the training images in python list

    - `image_tensor = sess.run(self.input)` : convert numpy array to tensor
    
    - `if(image_tensor.size == img_size*batch_size*img_layer)` : check the shapes are correct
    
    - `self.A_input[i] = image_tensor.reshape(image_shape)` : fill the tensor with an image one by one
    
        - individual image shape : (1, height, width, channel)
        

### setup models

In [ ]:
def model_setup(self):

        self.input_A = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_layer], 
                                      name="input_A")
        self.input_B = tf.placeholder(tf.float32, [batch_size, img_width, img_height, img_layer], 
                                      name="input_B")
        
        self.fake_pool_A = tf.placeholder(tf.float32, [None, img_width, img_height, img_layer], 
                                          name="fake_pool_A")
        self.fake_pool_B = tf.placeholder(tf.float32, [None, img_width, img_height, img_layer], 
                                          name="fake_pool_B")

        self.global_step = tf.Variable(0, name="global_step", trainable=False)

        self.num_fake_inputs = 0

        self.lr = tf.placeholder(tf.float32, shape=[], name="lr")

        with tf.variable_scope("Model") as scope:
            
            self.fake_B = build_generator_resnet_9blocks(self.input_A, name="g_A")
            # fake zebra <- real horse
            self.fake_A = build_generator_resnet_9blocks(self.input_B, name="g_B")
            # fake horse <- real zebra
            
            self.rec_A = build_gen_discriminator(self.input_A, "d_A")
            # is this horse real or fake? <- real horse
            self.rec_B = build_gen_discriminator(self.input_B, "d_B")
            # is this zebra real or fake? <- real zebra

            scope.reuse_variables()

            self.fake_rec_A = build_gen_discriminator(self.fake_A, "d_A")
            # is this horse real or fake? <- fake horse
            self.fake_rec_B = build_gen_discriminator(self.fake_B, "d_B")
            # is this zebra real or fake? <- fake zebra
            
            self.cyc_A = build_generator_resnet_9blocks(self.fake_B, "g_B")
            # cycled horse <- fake zebra
            self.cyc_B = build_generator_resnet_9blocks(self.fake_A, "g_A")
            # cycled zebra <- fake horse

            scope.reuse_variables()

            self.fake_pool_rec_A = build_gen_discriminator(self.fake_pool_A, "d_A")
            # is this horse real or fake? <- cycled horse
            self.fake_pool_rec_B = build_gen_discriminator(self.fake_pool_B, "d_B")
            # is this zebra real or fake? <- cycled zebra

- `self.input_A` / `self.input_B` : placeholder for training images (A: horse, B: zebra)

- `self.fake_A` / `self.fake_B` : Generated images by corresponding generator of input_A ($G$) and input_B ($F$)

- `self.lr` : Learning rate

</br>

**[Terms]**

- **A** : horse

    - **g_A** : generating fake zebra
    
    - **d_A** : discriminating horse is real
    
- **B** : zebra

    - **g_B** : generating fake horse
    
    - **d_B** : discriminating zebra is real


#### Model A (horse $\rightarrow$ zebra)

${L_{GAN}(G(x), y) + ||F(G(x)) - x||_{1}}$ $\Rightarrow$ ${L_{GAN}(Generated Fake Zebra, Real Zebra) + ||Cycled Horse - Real Horse||_{1}}$
>
> - $x$ : real horse image
>
> - $G$ : horse $\rightarrow$ zebra
>
>     - $G(x)$ : generated fake zebra
>
>
> - $F$ : zebra $\rightarrow$ horse
>
>     - $F(G(x))$ : cycled (returned) horse


1. generator $G(x)$ : **real horse** from training data $\rightarrow$ **fake zebra**

    - `self.fake_B = build_generator_resnet_9blocks(self.input_A, name="g_A")`
    

2. discriminator $D(y)$ : is this **zebra real or fake** $\leftarrow$ **real zebra**

    - `self.rec_B = build_gen_discriminator(self.input_B, "d_B")`
    

3. discriminator $D(G(x))$ : is this **zebra real or fake** $\leftarrow$ generated **fake zebra**

    - `self.fake_rec_B = build_gen_discriminator(self.fake_B, "d_B")`
    

4. generator $F(G(x))$ : generated **fake zebra** $\rightarrow$ **cycled horse**

    - `self.cyc_A = build_generator_resnet_9blocks(self.fake_B, "g_B")`


#### Model B (zebra $\rightarrow$ horse)

${L_{GAN}(F(y), x) + ||G(F(y)) - y||_{1}}$ $\Rightarrow$ ${L_{GAN}(Generated Fake Horse, Real Horse) + ||Cycled Zebra - Real Zebra||_{1}}$
>
> - $y$ : real zebra image
>
> - $F$ : zebra $\rightarrow$ horse
>
>     - $F(y)$ : generated fake horse
>
>
> - $G$ : horse $\rightarrow$ zebra
>
>     - $G(F(y))$ : cycled (returned) zebra


1. generator $G(x)$ : **real zebra** from training data $\rightarrow$ **fake horse**

     - `self.fake_A = build_generator_resnet_9blocks(self.input_B, name="g_B")`


2. discriminator $D(y)$ : is this **horse real or fake** $\leftarrow$ **real horse**

     - `self.rec_A = build_gen_discriminator(self.input_A, "d_A")`


3. discriminator $D(G(x))$ : is this **horse real or fake** $\leftarrow$ generated **fake horse**

     - `self.fake_rec_A = build_gen_discriminator(self.fake_A, "d_A")`


4. generator $F(G(x))$ : generated **fake horse** $\rightarrow$ **cycled zebra**

     - `self.cyc_B = build_generator_resnet_9blocks(self.fake_A, "g_A")`


### Loss

In [ ]:
def loss_calc(self):

        ''' In this function we are defining the variables for loss calcultions and traning model
        d_loss_A/d_loss_B -> loss for discriminator A/B
        g_loss_A/g_loss_B -> loss for generator A/B
        *_trainer -> Variaous trainer for above loss functions
        *_summ -> Summary variables for above loss functions'''

        cyc_loss = tf.reduce_mean(tf.abs(self.input_A-self.cyc_A)) + tf.reduce_mean(
            tf.abs(self.input_B-self.cyc_B))
        
        disc_loss_A = tf.reduce_mean(tf.squared_difference(self.fake_rec_A,1))
        disc_loss_B = tf.reduce_mean(tf.squared_difference(self.fake_rec_B,1))
        
        g_loss_A = cyc_loss*10 + disc_loss_B
        g_loss_B = cyc_loss*10 + disc_loss_A

        d_loss_A = (tf.reduce_mean(tf.square(self.fake_pool_rec_A)) + tf.reduce_mean(
            tf.squared_difference(self.rec_A,1))) / 2.0
        d_loss_B = (tf.reduce_mean(tf.square(self.fake_pool_rec_B)) + tf.reduce_mean(
            tf.squared_difference(self.rec_B,1))) / 2.0

        
        optimizer = tf.train.AdamOptimizer(self.lr, beta1=0.5)

        self.model_vars = tf.trainable_variables()

        d_A_vars = [var for var in self.model_vars if 'd_A' in var.name]
        g_A_vars = [var for var in self.model_vars if 'g_A' in var.name]
        d_B_vars = [var for var in self.model_vars if 'd_B' in var.name]
        g_B_vars = [var for var in self.model_vars if 'g_B' in var.name]
        
        self.d_A_trainer = optimizer.minimize(d_loss_A, var_list=d_A_vars)
        self.d_B_trainer = optimizer.minimize(d_loss_B, var_list=d_B_vars)
        self.g_A_trainer = optimizer.minimize(g_loss_A, var_list=g_A_vars)
        self.g_B_trainer = optimizer.minimize(g_loss_B, var_list=g_B_vars)

        for var in self.model_vars: print(var.name)

        #Summary variables for tensorboard

        self.g_A_loss_summ = tf.summary.scalar("g_A_loss", g_loss_A)
        self.g_B_loss_summ = tf.summary.scalar("g_B_loss", g_loss_B)
        self.d_A_loss_summ = tf.summary.scalar("d_A_loss", d_loss_A)
        self.d_B_loss_summ = tf.summary.scalar("d_B_loss", d_loss_B)

#### Least Square GANs

##### **[Model A]** (horse $\rightarrow$ zebra)

$L_{GAN}(G, D_{Y}, X, Y) = E_{y \sim P_{data}(y)}[(D_{Y}(y) - 1)^2] + E_{x \sim P_{data}(x)}[(D_{Y}(G(x)))^2]$

##### Model B (zebra $\rightarrow$ horse)

$L_{GAN}(F, D_{X}, X, Y) = E_{y \sim P_{data}(x)}[(D_{X}(x) - 1)^2] + E_{x \sim P_{data}(y)}[(D_{X}(F(y)))^2]$



- **[`cyc_loss`]** : L1 Loss (pixel level difference)

    - $||F(G(x)) - x||_{1}$
    
        - cycled horse $-$ real horse

        - `tf.reduce_mean(tf.abs(self.input_A-self.cyc_A))`
        
    - $||G(F(y)) - y||_{1}$
    
        - cycled zebra $-$ real zebra
        
        - `tf.reduce_mean(tf.abs(self.input_B-self.cyc_B))`



- **[`g_loss_A`]** / `g_loss_B` : Loss for **Generator** **[A ($G$)]** / B ($F$)

    - $||F(G(x)) - x||_{1} + ||G(F(y)) - y||_{1}$
    
        - L1 Loss
    
        - `cyc_loss`

    - $[(D_{Y}(G(x)) - 1)^2]$
    
        - make fake zebra image $G(x)$ be judged as real (1) as possible
    
        - `disc_loss_B = tf.reduce_mean(tf.squared_difference(self.fake_rec_B,1))`
        


- `d_loss_A` / **[`d_loss_B`]** : Loss for **Discriminator** A ($D_{X}$) / **[B ($D_{Y}$)]**

    - $[D_{Y}(G(x))^2]$
    
        - judge fake horse image as false (0) as possible
    
        - `tf.reduce_mean(tf.square(self.fake_pool_rec_B))`
        
    - $[(D_{Y}(y) - 1)^2]$
    
        - judge real horse image as real (1) as possible
    
        - `tf.reduce_mean(tf.squared_difference(self.rec_B,1))`
        
---

- `tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999)`

    - learning_rate : A Tensor or a floating point value. The learning rate

    - beta1 : The exponential decay rate for the 1st moment estimates (l1 regularization)

    - beta2 : The exponential decay rate for the 2nd moment estimates (l2 regularization)


- `tf.trainable_variables()`

    - Returns all variables created with `trainable = True`
    

- `tf.summary.scalar(name, scalar)`

    - Tensorboard
    

### save training results

In [ ]:
def save_training_images(self, sess, epoch):

        if not os.path.exists("./output/imgs"):
            os.makedirs("./output/imgs")

        for i in range(0,10):
            fake_A_temp, fake_B_temp, cyc_A_temp, cyc_B_temp = sess.run(
                [self.fake_A, self.fake_B, self.cyc_A, self.cyc_B],
                feed_dict={self.input_A:self.A_input[i], self.input_B:self.B_input[i]})
            imageio.imwrite("./output/imgs/fakeB_"+ str(epoch) + "_" + str(i)+".jpg",
                            ((fake_A_temp[0]+1)*127.5).astype(np.uint8))
            imageio.imwrite("./output/imgs/fakeA_"+ str(epoch) + "_" + str(i)+".jpg",
                            ((fake_B_temp[0]+1)*127.5).astype(np.uint8))
            imageio.imwrite("./output/imgs/cycA_"+ str(epoch) + "_" + str(i)+".jpg",
                            ((cyc_A_temp[0]+1)*127.5).astype(np.uint8))
            imageio.imwrite("./output/imgs/cycB_"+ str(epoch) + "_" + str(i)+".jpg",
                            ((cyc_B_temp[0]+1)*127.5).astype(np.uint8))
            imageio.imwrite("./output/imgs/inputA_"+ str(epoch) + "_" + str(i)+".jpg", 
                            ((self.A_input[i][0]+1)*127.5).astype(np.uint8))
            imageio.imwrite("./output/imgs/inputB_"+ str(epoch) + "_" + str(i)+".jpg",
                            ((self.B_input[i][0]+1)*127.5).astype(np.uint8))

- `os.makedirs(path)` : make save directory

- `from scipy.misc import imsave`

    - `imsave(path)` : save image in specific directory


### classifying generated fake images

In [ ]:
def fake_image_pool(self, num_fakes, fake, fake_pool):

        if(num_fakes < pool_size):
            fake_pool[num_fakes] = fake
            return fake
        else :
            p = random.random()
            if p > 0.5:
                random_id = random.randint(0,pool_size-1)
                temp = fake_pool[random_id]
                fake_pool[random_id] = fake
                return temp
            else :
                return fake

> This function saves the generated image to corresponding pool of images.
>
> - It keeps filling the pool untill it is full 
>
>     - `if(num_fakes < pool_size):`
>
>
> - and then randomly selects an already stored image 
>
>     - `p = random.random()`
>    
>     - `if p > 0.5:`
>
> - and replace it with new one

- `random.randint(min, max)` : pop random value between min and max value, inclusive


### training

In [ ]:
def train(self):


''' Training Function '''


# Load Dataset from the dataset folder
self.input_setup()  

#Build the network
self.model_setup()

#Loss function calculations
self.loss_calc()

# Initializing the global variables
init = [tf.global_variables_initializer(), tf.local_variables_initializer()]
saver = tf.train.Saver()     

with tf.Session() as sess:
    sess.run(init)

    #Read input to nd array
    self.input_read(sess)

    #Restore the model to run the model from last checkpoint
    if to_restore:
        chkpt_fname = tf.train.latest_checkpoint(check_dir)
        saver.restore(sess, chkpt_fname)

    writer = tf.summary.FileWriter("./output/2")

    if not os.path.exists(check_dir):
        os.makedirs(check_dir)

    # Training Loop
    for epoch in range(sess.run(self.global_step),100):                
        print ("In the epoch ", epoch)
        saver.save(sess,os.path.join(check_dir,"cyclegan"),global_step=epoch)

        # Dealing with the learning rate as per the epoch number

        if(epoch < 100) :
            curr_lr = 0.0002
        else:
            # learning rate decay
            curr_lr = 0.0002 - 0.0002*(epoch-100)/100

        # ------------------------------------------------------

        if(save_training_images):
            self.save_training_images(sess, epoch)

        # sys.exit()

        for ptr in range(0,max_images):
            print("In the iteration ",ptr)
            print("Starting",time.time()*1000.0)

            # Optimizing the G_A network--------------------------------------------------------------

            _, fake_B_temp, summary_str = sess.run([self.g_A_trainer, self.fake_B, self.g_A_loss_summ],
                                                   feed_dict={self.input_A:self.A_input[ptr],
                                                              self.input_B:self.B_input[ptr], 
                                                              self.lr:curr_lr})

            writer.add_summary(summary_str, epoch*max_images + ptr)                    
            fake_B_temp1 = self.fake_image_pool(self.num_fake_inputs, fake_B_temp, self.fake_images_B)

            # ----------------------------------------------------------------------------------------

            # Optimizing the D_B network--------------------------------------------------------------

            _, summary_str = sess.run([self.d_B_trainer, self.d_B_loss_summ],
                                      feed_dict={self.input_A:self.A_input[ptr], 
                                                 self.input_B:self.B_input[ptr], 
                                                 self.lr:curr_lr, 
                                                 self.fake_pool_B:fake_B_temp1})
            writer.add_summary(summary_str, epoch*max_images + ptr)

            # ----------------------------------------------------------------------------------------

            # Optimizing the G_B network--------------------------------------------------------------

            _, fake_A_temp, summary_str = sess.run([self.g_B_trainer, self.fake_A, self.g_B_loss_summ],
                                                   feed_dict={self.input_A:self.A_input[ptr],
                                                              self.input_B:self.B_input[ptr], 
                                                              self.lr:curr_lr})

            writer.add_summary(summary_str, epoch*max_images + ptr)

            fake_A_temp1 = self.fake_image_pool(self.num_fake_inputs, fake_A_temp, self.fake_images_A)

            # ----------------------------------------------------------------------------------------

            # Optimizing the D_A network--------------------------------------------------------------

            _, summary_str = sess.run([self.d_A_trainer, self.d_A_loss_summ],
                                      feed_dict={self.input_A:self.A_input[ptr], 
                                                 self.input_B:self.B_input[ptr], 
                                                 self.lr:curr_lr, 
                                                 self.fake_pool_A:fake_A_temp1})

            writer.add_summary(summary_str, epoch*max_images + ptr)

            # ----------------------------------------------------------------------------------------

            self.num_fake_inputs+=1                       

        sess.run(tf.assign(self.global_step, epoch + 1))

    writer.add_graph(sess.graph)

### testing

In [ ]:
def test(self):


    ''' Testing Function'''

    print("Testing the results")

    self.input_setup()

    self.model_setup()
    saver = tf.train.Saver()
    init = [tf.global_variables_initializer(), tf.local_variables_initializer()]

    with tf.Session() as sess:

        sess.run(init)

        self.input_read(sess)

        chkpt_fname = tf.train.latest_checkpoint(check_dir)
        saver.restore(sess, chkpt_fname)

        if not os.path.exists("./output/imgs/test/"):
            os.makedirs("./output/imgs/test/")            

        for i in range(0,100):
            fake_A_temp, fake_B_temp = sess.run([self.fake_A, self.fake_B],
                                                feed_dict={self.input_A:self.A_input[i],
                                                           self.input_B:self.B_input[i]})
            imageio.imwrite("./output/imgs/test/fakeB_"+str(i)+".jpg",
                            ((fake_A_temp[0]+1)*127.5).astype(np.uint8))
            imageio.imwrite("./output/imgs/test/fakeA_"+str(i)+".jpg",
                            ((fake_B_temp[0]+1)*127.5).astype(np.uint8))
            imageio.imwrite("./output/imgs/test/inputA_"+str(i)+".jpg",
                            ((self.A_input[i][0]+1)*127.5).astype(np.uint8))
            imageio.imwrite("./output/imgs/test/inputB_"+str(i)+".jpg",
                            ((self.B_input[i][0]+1)*127.5).astype(np.uint8))

- **restore previously trained checkpoint**

```python
chkpt_fname = tf.train.latest_checkpoint(check_dir)
saver.restore(sess, chkpt_fname)
```

- **generate fake images (desired output)**

```python
fake_A_temp, fake_B_temp = sess.run([self.fake_A, self.fake_B],
                                    feed_dict={self.input_A:self.A_input[i],
                                               self.input_B:self.B_input[i]})
```

### main

In [ ]:
def main():
    
    model = CycleGAN()
    if to_train:
        model.train()
    elif to_test:
        model.test()